In [1]:
# Install required libraries
!pip install librosa transformers

In [2]:
# Define paths to data
cha_path = '/kaggle/input/pittcombined/PittCombined/cha'
mp3_path = '/kaggle/input/pittcombined/PittCombined/mp3'

In [3]:
# Import necessary libraries
import os
import librosa
import pandas as pd
from transformers import BertTokenizer, BertModel


In [4]:
# List files in directories
cha_files = [os.path.join(cha_path, file) for file in os.listdir(cha_path) if file.endswith('.cha')]
mp3_files = [os.path.join(mp3_path, file) for file in os.listdir(mp3_path) if file.endswith('.mp3')]

print(f"Found {len(cha_files)} CHA files and {len(mp3_files)} MP3 files.")

Found 1255 CHA files and 1253 MP3 files.


In [5]:
import os
import re
import random
from collections import defaultdict, Counter

def extract_diagnosis(cha_file):
    """Extracts the diagnosis from a CHAT transcript file."""
    with open(cha_file, 'r') as file:
        content = file.read()
    match = re.search(r'@ID:\s*[^|]*\|[^|]*\|[^|]*\|[^|]*\|[^|]*\|([^|]*)\|', content)
    return match.group(1) if match else None

# Paths to your CHA and MP3 files
cha_path = '/kaggle/input/pittcombined/PittCombined/cha'
mp3_path = '/kaggle/input/pittcombined/PittCombined/mp3'

# Load all CHA files and their diagnoses
cha_files = [f for f in os.listdir(cha_path) if f.endswith('.cha')]
diagnoses = [extract_diagnosis(os.path.join(cha_path, f)) for f in cha_files]

# Group files by diagnosis, ensuring corresponding MP3 exists
files_by_diagnosis = defaultdict(list)
for cha_file, diag in zip(cha_files, diagnoses):
    mp3_file = cha_file.replace('.cha', '.mp3')
    if os.path.exists(os.path.join(mp3_path, mp3_file)):
        files_by_diagnosis[diag].append(cha_file)

# Select up to 500 files, trying to balance across diagnoses
selected_files = []
for diag, files in files_by_diagnosis.items():
    select_count = min(len(files), max(500 // len(files_by_diagnosis), 1))
    selected_files.extend(random.sample(files, select_count))

# Ensure the selection does not exceed 10 if categories were unbalanced
selected_files = random.sample(selected_files, min(500, len(selected_files)))

# Count of selected diagnoses
selected_diagnoses = [extract_diagnosis(os.path.join(cha_path, f)) for f in selected_files]
diagnosis_count = Counter(selected_diagnoses)

# Find corresponding MP3 files
selected_mp3_files = [f.replace('.cha', '.mp3') for f in selected_files]

# Collect full paths for the selected files
cha_files = [os.path.join(cha_path, f) for f in selected_files]
mp3_files = [os.path.join(mp3_path, f) for f in selected_mp3_files]

# Output results
print("Selected CHA files:", cha_files)
print("Selected MP3 files:", mp3_files)
print("Diagnosis counts:", dict(diagnosis_count))


Selected CHA files: ['/kaggle/input/pittcombined/PittCombined/cha/511-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/598-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/056-3.cha', '/kaggle/input/pittcombined/PittCombined/cha/016-0r.cha', '/kaggle/input/pittcombined/PittCombined/cha/267-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/631-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/296-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/221-3f.cha', '/kaggle/input/pittcombined/PittCombined/cha/362-1.cha', '/kaggle/input/pittcombined/PittCombined/cha/270-1f.cha', '/kaggle/input/pittcombined/PittCombined/cha/042-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/563-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/113-1.cha', '/kaggle/input/pittcombined/PittCombined/cha/672-0r.cha', '/kaggle/input/pittcombined/PittCombined/cha/236-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/120-1s.cha', '/kaggle/input/pittcombined/PittCombined/cha/707-0.cha', '/ka

In [6]:
cha_base_names = set([os.path.splitext(os.path.basename(f))[0] for f in cha_files])
mp3_base_names = set([os.path.splitext(os.path.basename(f))[0] for f in mp3_files])

unmatched_cha = cha_base_names - mp3_base_names
unmatched_mp3 = mp3_base_names - cha_base_names

print("Unmatched CHA files:", unmatched_cha)
print("Unmatched MP3 files:", unmatched_mp3)

Unmatched CHA files: set()
Unmatched MP3 files: set()


In [7]:
import librosa
from transformers import BertTokenizer, BertModel
import soundfile as sf

# Initialize Clinical BERT
tokenizer = BertTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = BertModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
import os
import librosa
import soundfile as sf

def preprocess_audio(mp3_file_path, output_wav_path):
    # Define output path within the writable directory
    output_wav_path = os.path.join('/kaggle/working', os.path.basename(output_wav_path))
    
    try:
        # Check if the WAV file already exists
        if not os.path.exists(output_wav_path):
            # Convert MP3 to WAV
            y, sr = librosa.load(mp3_file_path, sr=None)
            sf.write(output_wav_path, y, sr)
            print(f"Converted {mp3_file_path} to WAV.")
        else:
            print(f"WAV file already exists: {output_wav_path}")

        # Extract MFCC features
        y, sr = librosa.load(output_wav_path, sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        return mfccs.mean(axis=1)
    except Exception as e:
        print(f"An error occurred while processing {mp3_file_path}: {str(e)}")
        return None

# Replace 'mp3_files' with the actual list of mp3 file paths
audio_features = [preprocess_audio(f, f.replace('.mp3', '.wav')) for f in mp3_files if f.endswith('.mp3')]


Converted /kaggle/input/pittcombined/PittCombined/mp3/511-0.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/598-0f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/056-3.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/016-0r.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/267-2.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/631-0.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/296-0.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/221-3f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/362-1.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/270-1f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/042-2.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/563-0.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/113-1.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/672-0r

In [9]:
# Define a function to preprocess text data
def preprocess_text(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    # Insert text cleaning code here as necessary
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    outputs = model(**inputs)
    return outputs.last_hidden_state.squeeze().detach().numpy()

In [10]:
print(cha_files)

['/kaggle/input/pittcombined/PittCombined/cha/511-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/598-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/056-3.cha', '/kaggle/input/pittcombined/PittCombined/cha/016-0r.cha', '/kaggle/input/pittcombined/PittCombined/cha/267-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/631-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/296-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/221-3f.cha', '/kaggle/input/pittcombined/PittCombined/cha/362-1.cha', '/kaggle/input/pittcombined/PittCombined/cha/270-1f.cha', '/kaggle/input/pittcombined/PittCombined/cha/042-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/563-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/113-1.cha', '/kaggle/input/pittcombined/PittCombined/cha/672-0r.cha', '/kaggle/input/pittcombined/PittCombined/cha/236-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/120-1s.cha', '/kaggle/input/pittcombined/PittCombined/cha/707-0.cha', '/kaggle/input/pittcombi

In [11]:
# Process each file
text_embeddings = [preprocess_text(f) for f in cha_files]

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming text_embeddings and audio_features are lists of numpy arrays
# Convert lists to numpy arrays
text_features = np.array(text_embeddings)
audio_features = np.stack(audio_features)

In [13]:
print(text_features)

[[[-1.35623202e-01  2.14709833e-01 -7.75165915e-01 ...  3.35970998e-01
    3.51336896e-01 -8.00927654e-02]
  [-3.89657557e-01  3.51313353e-02  1.85759179e-02 ... -1.09697253e-01
    5.92928708e-01 -5.89498341e-01]
  [ 1.70005351e-01 -1.34050295e-01  3.81877810e-01 ...  6.75277829e-01
   -5.41085482e-01  4.12126452e-01]
  ...
  [-2.54750699e-01 -4.35775220e-01  1.04881845e-01 ...  2.08811045e-01
    2.98727781e-01 -4.44821298e-01]
  [-3.17106664e-01 -4.32794392e-01  2.46873632e-01 ...  6.78334951e-01
   -2.17052594e-01  5.16788960e-01]
  [ 6.07613385e-01 -7.19740748e-01 -7.13752806e-01 ...  6.88725948e-01
    1.21106923e+00  5.07907569e-01]]

 [[ 9.24162008e-03  9.03383791e-02 -6.81193292e-01 ...  2.70205230e-01
   -1.77126765e-01 -9.31924209e-02]
  [-3.99692655e-01  2.26654679e-01  1.45657986e-01 ...  1.25439212e-01
    4.58804786e-01 -4.79421288e-01]
  [ 1.86581090e-01 -2.11837292e-01  5.09587467e-01 ...  7.75573492e-01
   -3.58785391e-01  2.69229740e-01]
  ...
  [-2.35267133e-01 -1.1

In [14]:
print(audio_features)

[[-4.61405579e+02  1.10085884e+02  1.05078030e+01 ... -8.57927084e-01
  -2.99813080e+00 -3.97093511e+00]
 [-3.75353363e+02  1.33806625e+02  6.38955641e+00 ... -9.56310391e-01
  -1.76106668e+00 -3.62203312e+00]
 [-4.93226471e+02  1.27623306e+02  3.59414825e+01 ... -9.31636143e+00
   4.27531147e+00 -1.30359519e+00]
 ...
 [-3.95028534e+02  1.16160080e+02  2.20964203e+01 ... -9.45268333e-01
  -1.19022524e+00 -1.05495119e+00]
 [-4.36144653e+02  1.08254410e+02 -4.32961911e-01 ... -8.01813030e+00
   3.63161135e+00 -3.72248650e-01]
 [-3.92434967e+02  1.28824509e+02 -2.19194946e+01 ... -5.04412413e+00
  -8.21580172e-01 -1.28705513e+00]]


In [15]:
text_features = np.mean(text_features, axis=1)


In [16]:
print(text_features)

[[-0.18517588 -0.31501812 -0.22222951 ...  0.07503347  0.11869515
  -0.07724011]
 [-0.03952688 -0.0916104  -0.1897387  ...  0.25596625  0.04222545
  -0.09613381]
 [-0.13370895 -0.27838337 -0.22590096 ...  0.07133634  0.05939862
  -0.10254636]
 ...
 [-0.02482527 -0.2569128  -0.32036158 ...  0.26925686 -0.11895912
  -0.179712  ]
 [-0.15692224 -0.27425054 -0.1816842  ... -0.02751     0.16104476
  -0.10412122]
 [-0.17372383 -0.27048343 -0.13540523 ...  0.02846383  0.0351217
  -0.09402069]]


In [17]:
# Ensure audio_features is 2D (it should already be if you've extracted features correctly)
if audio_features.ndim > 2:
    audio_features = np.mean(audio_features, axis=1)

In [18]:
# Combine text and audio features
combined_features = np.concatenate([text_features, audio_features], axis=1)

In [19]:
print(combined_features)

[[-0.18517588 -0.31501812 -0.22222951 ... -0.8579271  -2.9981308
  -3.970935  ]
 [-0.03952688 -0.0916104  -0.1897387  ... -0.9563104  -1.7610667
  -3.622033  ]
 [-0.13370895 -0.27838337 -0.22590096 ... -9.316361    4.2753115
  -1.3035952 ]
 ...
 [-0.02482527 -0.2569128  -0.32036158 ... -0.94526833 -1.1902252
  -1.0549512 ]
 [-0.15692224 -0.27425054 -0.1816842  ... -8.01813     3.6316113
  -0.37224865]
 [-0.17372383 -0.27048343 -0.13540523 ... -5.044124   -0.8215802
  -1.2870551 ]]


In [20]:
# Normalize the combined features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(combined_features)

In [21]:
import re

def extract_diagnostic_code(cha_file):
    # Read the content of the .cha file
    with open(cha_file, 'r') as file:
        content = file.read()

    # Regex to find the diagnosis in the file content based on the updated structure
    match = re.search(r'@ID:\s*[^|]*\|[^|]*\|[^|]*\|[^|]*\|[^|]*\|([^|]*)\|\|', content)
    if match:
        return match.group(1).strip()
    return None

# Assuming cha_files is a list of paths to your .cha files
labels = [extract_diagnostic_code(f) for f in cha_files]

# Example of how you might use this
for label in labels[:10]:  # Print first 10 labels to verify
    print(label)


PossibleAD
ProbableAD
Control
MCI
Control
Control
Control
MCI
ProbableAD
ProbableAD


In [22]:
from sklearn.preprocessing import LabelEncoder

# Encode labels as integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


In [23]:
print(encoded_labels)

[6 8 1 3 1 1 1 3 8 8 1 8 1 6 6 3 6 3 6 6 4 6 1 8 3 3 9 8 1 1 3 6 8 8 1 1 8
 6 1 1 8 1 1 6 4 9 1 6 8 9 8 3 1 1 8 1 9 1 1 3 3 8 8 6 3 6 3 9 9 8 7 6 8 6
 8 3 1 3 3 3 8 7 3 3 8 1 8 8 8 9 3 6 4 8 3 6 1 9 1 6 1 4 1 3 8 8 6 1 6 1 1
 1 8 6 3 9 1 1 9 9 6 6 8 9 3 3 6 6 6 6 1 8 7 5 1 6 3 1 4 3 6 8 8 1 1 3 1 4
 8 3 3 3 6 6 3 5 1 3 1 3 6 8 8 6 9 8 6 6 3 1 6 6 4 3 1 4 6 3 8 6 3 8 3 8 3
 6 1 6 2 8 8 1 8 8 6 5 5 9 6 3 3 6 1 6 8 8 8 6 1 9 9 8 3 3 3 1 6 6 8 6 1 3
 3 1 4 6 1 3 3 9 3 1 8 0 4 1 8 1 6 6 8 1 6 4 3 6 7 4 8 6 3 3 9 3 3 3 8 9 8
 8]


In [24]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Conv1D, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam

2024-08-13 19:31:48.548931: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 19:31:48.549181: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 19:31:48.735797: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [26]:
# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(normalized_features, encoded_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Finding the maximum label value across all datasets to ensure consistency
max_label = max(np.max(y_train), np.max(y_val), np.max(y_test)) + 1  # Plus one because classes are zero-indexed

# One-hot encode the labels with a consistent number of classes across datasets
y_train = to_categorical(y_train, num_classes=max_label)
y_val = to_categorical(y_val, num_classes=max_label)
y_test = to_categorical(y_test, num_classes=max_label)




In [27]:
print("y_test",y_test)

y_test [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0

In [28]:
# Model architecture
input_text = Input(shape=(768,))  # Text features from Clinical BERT
input_audio = Input(shape=(13,))  # Audio features, assuming MFCCs with 13 coefficients

# Text pathway
text_dense = Dense(128, activation='relu')(input_text)
text_out = Dropout(0.2)(text_dense)


In [29]:
from tensorflow.keras.layers import Reshape

# Audio pathway
audio_reshape = Reshape((13, 1))(input_audio)
conv1 = Conv1D(64, kernel_size=3, activation='relu')(audio_reshape)
conv1_bn = BatchNormalization()(conv1)
conv1_pool = GlobalAveragePooling1D()(conv1_bn)
audio_out = Dropout(0.2)(conv1_pool)

In [30]:
# Fusion and output
concatenated = Concatenate()([text_out, audio_out])
dense_layer = Dense(64, activation='relu')(concatenated)
# Assuming y_train has already been one-hot encoded correctly
num_classes = y_train.shape[1]

# Adjust your output layer
output_layer = Dense(num_classes, activation='softmax')(dense_layer)

In [31]:
print(concatenated)

<KerasTensor shape=(None, 192), dtype=float32, sparse=False, name=keras_tensor_9>


In [32]:
from tensorflow.keras.metrics import Precision, Recall

# Compile model
model = Model(inputs=[input_text, input_audio], outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', 
              metrics=['accuracy', Precision(), Recall()])

In [33]:
# Fit model on training data
model.fit([X_train[:, :768], X_train[:, 768:]], y_train, validation_data=([X_val[:, :768], X_val[:, 768:]], y_val), epochs=100, batch_size=32)

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - accuracy: 0.1021 - loss: 2.7635 - precision: 0.1740 - recall: 0.0211 - val_accuracy: 0.3077 - val_loss: 2.0339 - val_precision: 0.3750 - val_recall: 0.1154
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3681 - loss: 1.8313 - precision: 0.5996 - recall: 0.1704 - val_accuracy: 0.3654 - val_loss: 1.7585 - val_precision: 0.5000 - val_recall: 0.2500
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4943 - loss: 1.4108 - precision: 0.7077 - recall: 0.2759 - val_accuracy: 0.4423 - val_loss: 1.6000 - val_precision: 0.5417 - val_recall: 0.2500
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5480 - loss: 1.2629 - precision: 0.5966 - recall: 0.3187 - val_accuracy: 0.4038 - val_loss: 1.5869 - val_precision: 0.5417 - val_recall: 0.2500
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6265 - loss: 1.0376 - precision: 0.7674 - recall: 0.4288 - val_accuracy: 0.3654 - val_loss: 1.6371 - va

In [34]:
# Evaluate the model on the test set
evaluation = model.evaluate([X_test[:, :768], X_test[:, 768:]], y_test)
print(f'Accuracy: {evaluation[1]*100:.2f}%, Precision: {evaluation[2]*100:.2f}%, Recall: {evaluation[3]*100:.2f}%')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5256 - loss: 2.7618 - precision: 0.5267 - recall: 0.5128 
Accuracy: 53.85%, Precision: 54.00%, Recall: 51.92%
